# I. Entity Linking

## 1) Spotlight

In [7]:
with open('allText.txt', encoding='utf-8') as f:
    text = f.read()

In [1]:
short_text = 'Barack Obama was the president of the United States'

In [2]:
import requests
result = requests.post("http://localhost:2225/rest/annotate", data=("text=" + short_text).encode('UTF-8'), headers={'Accept': 'text/turtle'})

In [3]:
# La réponse doit etre [200]
result

<Response [200]>

In [4]:
print(result.text)

@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
@prefix itsrdf: <http://www.w3.org/2005/11/its/rdf#> .
@prefix nif:   <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#> .

<http://www.dbpedia-spotlight.com/#offset_0_12>
        a                       nif:OffsetBasedString , nif:Phrase ;
        nif:anchorOf            "Barack Obama" ;
        nif:beginIndex          "0"^^xsd:nonNegativeInteger ;
        nif:endIndex            "12"^^xsd:nonNegativeInteger ;
        nif:referenceContext    <http://www.dbpedia-spotlight.com/#offset_0_51> ;
        itsrdf:taAnnotatorsRef  <http://www.dbpedia-spotlight.com> ;
        itsrdf:taClassRef       <http://www.wikidata.org/entity/Q5> , <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson> , <http://dbpedia.org/ontology/Person> , <http://dbpedia.org/ontology/Eukaryote> , <Http://xmlns.com/foaf/0.1/Person> , <http://www.wikidata.org/entity/Q19088> , <http://www.wikidata.org/entity/Q215627> , <http://dbpedia.org/o

In [45]:
# La longueur du texte est tres grande, donc on va le diviser en phrases
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)

len(sentences)

2718

In [46]:
results = []
for sen in sentences:
    results.append(requests.post("http://localhost:2222/rest/annotate", data=("text=" + sen).encode('UTF-8'), headers={'Accept': 'text/turtle'}))

In [49]:
print(sentences[0] + '\n' + '*'*100 + '\n' + results[0].text)

 
FICHE MÉMO 
Prise en charge du patient présentant 
une lombalgie commune 
 
 
RAPPORT D’ÉLABORATION 
Mars 2019  
Ce document n’est pas le texte de recommandations 
 
 
 
 
 
  Prise en charge du patient présentant une lombalgie commune 
La  méthode  d’élaboration  des  fiches  mémo  est  une  méthode  pour  produire  des 
recommandations ou messages-clés dans un temps court (6 mois environ) et dans un format 
court (recto-verso).
****************************************************************************************************
@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
@prefix itsrdf: <http://www.w3.org/2005/11/its/rdf#> .
@prefix nif:   <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#> .

<http://www.dbpedia-spotlight.com/#offset_57_66>
        a                       nif:Phrase , nif:OffsetBasedString ;
        nif:anchorOf            "lombalgie" ;
        nif:beginIndex          "57"^^xsd:nonNegativeInteger ;
        nif:endIndex            "66"^^xsd:no

In [49]:
# On va concatener tous les résultats donnés de chaque phrase
SpLt_Annotation = ""
for r in results:
    SpLt_Annotation += r.text

In [53]:
with open('SpLt_Annotation.ttl', 'w', encoding='utf8') as f:
    f.write(SpLt_Annotation)

In [9]:
with open('SpLt_Annotation.ttl', 'r', encoding='utf8') as f:
    SpLt_Annotation = f.read()

## 2) Entity Fishing

In [73]:
from nerd import nerd_client
client = nerd_client.NerdClient()

In [ ]:
annotation = client.disambiguate_text(
    text
)

In [6]:
type(annotation)

tuple

In [ ]:
with open('EntityFishingAnnotation.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(annotation))

In [12]:
with open('EntityFishingAnnotation.json') as f:
     entFich = json.loads(f.read())

In [8]:
entFich[1]

200

In [13]:
entities = entFich[0]['entities']
len(entFich[0]['entities'])

5415

In [42]:
entities[:3]

[{'rawName': 'lombalgie',
  'offsetStart': 57,
  'offsetEnd': 66,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'lombalgie',
  'offsetStart': 220,
  'offsetEnd': 229,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'format court',
  'offsetStart': 407,
  'offsetEnd': 420,
  'nerd_score': 1,
  'nerd_selection_score': 0.5137,
  'wikipediaExternalRef': 54180,
  'wikidataId': 'Q24862',
  'domains': ['Electronics', 'Computer_Science']}]

In [24]:
with open('EF_Annotation.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(entities))

In [2]:
with open('EF_Annotation.json', 'r', encoding='utf-8') as f:
    ef_Annotation = json.loads(f.read())

In [3]:
ef_Annotation

[{'rawName': 'lombalgie',
  'offsetStart': 57,
  'offsetEnd': 66,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'lombalgie',
  'offsetStart': 220,
  'offsetEnd': 229,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'format court',
  'offsetStart': 407,
  'offsetEnd': 420,
  'nerd_score': 1,
  'nerd_selection_score': 0.5137,
  'wikipediaExternalRef': 54180,
  'wikidataId': 'Q24862',
  'domains': ['Electronics', 'Computer_Science']},
 {'rawName': 'professionnel de santé',
  'offsetStart': 598,
  'offsetEnd': 622,
  'nerd_score': 1,
  'nerd_selection_score': 0.5329,
  'wikipediaExternalRef': 272680,
  'wikidataId': 'Q15220429',
  'domains': ['Health']},
 {'rawName': 'HAS',
  'type': 'BUSINESS',
  'offsetStart': 932,
  'offsetEnd'

# II. Analysis of overlap of extracted terms with Wikidata and DBpedia

## 1) Creation du dictionnaire de la terminologie avec les offsets
(On va utiliser un outil de reconnaissance de concepts qui indexe un dictionnaire TSV)

In [1]:
from os.path import join
import json

In [2]:
path = r'C:\Users\Anas\Desktop\M2SI\S4\Terminology Extraction'

In [7]:
with open(join(path,'termsuite_similaire.json')) as f:
    new_termsuite = json.loads(f.read())
len(new_termsuite)

1897

In [8]:
with open(join(path,'termsuite_unique.json')) as f:
    termsuite_unique = json.loads(f.read())
len(termsuite_unique)

2083

In [9]:
with open(join(path,'multilingual_unique.json')) as f:
    multilingual_unique = json.loads(f.read())
len(multilingual_unique)

3879

In [6]:
allTerms = new_termsuite + termsuite_unique + multilingual_unique
len(allTerms)

7859

In [2]:
with open('medicalTerms.json') as f:
    medicalTerms = json.loads(f.read())

In [34]:
len(medicalTerms)

3629

In [4]:
with open("dictionary_medical.tsv", "w", encoding="utf-8") as f:
    for i,term in enumerate(medicalTerms):
        f.write(str(i+1) + "\t" + term + "\n")

In [5]:
from pyclinrec.dictionary import MgrepDictionaryLoader
loader = MgrepDictionaryLoader("dictionary_medical.tsv")

from pyclinrec.recognizer import IntersStemConceptRecognizer


recognizer = IntersStemConceptRecognizer(dictionary_loader=loader,
                                         stop_words_file="data/stopwordsfr.txt",                                 
                                         termination_terms_file="data/termination_termsfr.txt")
recognizer.initialize()

Now loading the dictionary...


3628it [00:00, 37108.86it/s]


Now indexing the dictionary...


100%|██████████| 3628/3628 [00:13<00:00, 266.09it/s]


In [8]:
dict_offsets = []
annotations = recognizer.match_mentions(text)
for annotation in annotations[2]:
    concept_id = annotation.concept_id # The unique identifier of the matching concept as defined in the dictionary
    start = annotation.start # Start character offset of the annotation
    end = annotation.end # End character offset of the annotation
    matched_text = annotation.matched_text # The surface form of the text matching te annotation
    print(f"C={concept_id} [{start}:{end}] {matched_text}")
    dict_offsets.append((matched_text, start, end))

C=980 [367119:367125] impact
C=1912 [434571:434578] conseil
C=2235 [378998:379006] parcours
C=2662 [371114:371127] questionnaire
C=900 [400410:400425] Prise en charge
C=246 [388922:388930] physique
C=3045 [321393:321394] P
C=1917 [1773:1789] niveau de preuve
C=1913 [38896:38902] cancer
C=3257 [395302:395311] onset low
C=2565 [439395:439414] New England Journal
C=1128 [248820:248826] global
C=2887 [415616:415630] recommandation
C=907 [162450:162457] douleur
C=1003 [456304:456312] patients
C=2409 [345839:345849] lombalgies
C=2024 [82708:82712] Mars
C=1107 [434571:434578] conseil
C=2785 [144335:144341] taille
C=1032 [273160:273179] lombalgie chronique
C=2996 [327493:327501] imagerie
C=197 [242300:242306] Toward
C=2630 [69681:69691] Population
C=460 [333968:333970] WS
C=1290 [75132:75140] proposer
C=2311 [402815:402820] champ
C=174 [394918:394930] comportement
C=1791 [262506:262511] Grade
C=3130 [102645:102660] Recommandations
C=1821 [222079:222086] groupes
C=2327 [328534:328543] mouvement

In [33]:
len(dict_offsets)

33563

In [76]:
with open('dict_offsets.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list(set(dict_offsets))))

In [3]:
with open('dict_offsets.json', 'r', encoding='utf-8') as f:
    dict_offsets = json.loads(f.read())
len(dict_offsets)

27919

## 2) Mots communs entre les entités de Entity Fishing et le dictionnaire

### 1. Chargement des entités de Entity Fishing avec les offsets correspondants

In [2]:
import json
with open('EF_Annotation.json', 'r', encoding='utf-8') as f:
    ef_Annotation = json.loads(f.read())

In [3]:
ef_Annotation

[{'rawName': 'lombalgie',
  'offsetStart': 57,
  'offsetEnd': 66,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'lombalgie',
  'offsetStart': 220,
  'offsetEnd': 229,
  'nerd_score': 1,
  'nerd_selection_score': 0.6988,
  'wikipediaExternalRef': 250670,
  'wikidataId': 'Q852163',
  'domains': ['Medicine', 'Psychiatry', 'Health']},
 {'rawName': 'format court',
  'offsetStart': 407,
  'offsetEnd': 420,
  'nerd_score': 1,
  'nerd_selection_score': 0.5137,
  'wikipediaExternalRef': 54180,
  'wikidataId': 'Q24862',
  'domains': ['Electronics', 'Computer_Science']},
 {'rawName': 'professionnel de santé',
  'offsetStart': 598,
  'offsetEnd': 622,
  'nerd_score': 1,
  'nerd_selection_score': 0.5329,
  'wikipediaExternalRef': 272680,
  'wikidataId': 'Q15220429',
  'domains': ['Health']},
 {'rawName': 'HAS',
  'type': 'BUSINESS',
  'offsetStart': 932,
  'offsetEnd'

In [8]:
ef_offset_graph = []
ef_offset = []

for dic in ef_Annotation:
    if 'wikidataId' in dic.keys():
        ef_offset_graph.append((dic['rawName'] ,dic['offsetStart'], dic['offsetEnd'], 'http://www.wikidata.org/entity/' + dic['wikidataId']))
    ef_offset.append((dic['rawName'] ,dic['offsetStart'], dic['offsetEnd']))

In [5]:
len(ef_offset)

5415

In [6]:
ef_offset

[('lombalgie', 57, 66),
 ('lombalgie', 220, 229),
 ('format court', 407, 420),
 ('professionnel de santé', 598, 622),
 ('HAS', 932, 935),
 ('recherche documentaire', 1030, 1052),
 ('méta-analyse', 1313, 1325),
 ('B preuve', 1538, 1546),
 ('études de cohorte', 1691, 1708),
 ('études cas-témoins', 1801, 1819),
 ('Haute Autorité de Santé', 2383, 2406),
 ('Stade de France', 2459, 2474),
 ('Saint-Denis La Plaine', 2485, 2506),
 ('Fax', 2544, 2547),
 ('Haute Autorité de Santé', 2618, 2641),
 ('Haute Autorité de Santé', 2659, 2682),
 ('HAS', 2693, 2696),
 ('bonnes pratiques', 2711, 2727),
 ('lombalgie', 2804, 2813),
 ('Table des matières', 2823, 2841),
 ('recherche documentaire', 3176, 3198),
 ('lombalgie', 3533, 3542),
 ('lombalgie', 3713, 3722),
 ('Évaluation du risque', 4146, 4166),
 ('Prise en charge kinésithérapique', 4575, 4607),
 ('prévention secondaire', 5153, 5174),
 ('lombalgie', 5208, 5217),
 ('parties prenantes', 5272, 5289),
 ('parties prenantes', 5382, 5399),
 ('HAS', 5678, 5681

In [30]:
with open('ef_offset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(ef_offset)))

In [9]:
with open('ef_offset_graph.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(ef_offset_graph)))

In [10]:
with open('ef_offset.json', 'r', encoding='utf-8') as f:
    ef_offset = json.loads(f.read())

### 2. Recherche des entités communs avec les mêmes offsets (Comparaison Exacte), des entités de L'Entity Fishing qui représentent juste une partie des entités du dictionnaire et des entités de L'Entity Fishing qui englobent des entités du dictionnaire (Comparaision Partielle):

##### 1) Calcul du recouvrement exact :

In [11]:
ef_dict_com = []
for ef_mot in ef_offset:
    for dic_mot in dict_offsets:
        # Entités communs avec les mêmes offsets
        if (ef_mot[1] == dic_mot[1]) and (ef_mot[2] == dic_mot[2]):
            if ef_mot not in ef_dict_com:
                ef_dict_com.append(ef_mot[0])

##### 1) Calcul du recouvrement des matches partiels :

In [39]:
ef_dict_com_par = []
for ef_mot in ef_offset:
    for dic_mot in dict_offsets:
        # Entités du dictionnaire qui représentent juste une partie des entités de Spotlight
        if ((ef_mot[1] == dic_mot[1]) and (ef_mot[2] > dic_mot[2])) or ((ef_mot[1] < dic_mot[1]) and (ef_mot[2] == dic_mot[2])) or ((ef_mot[1] < dic_mot[1]) and (ef_mot[2] > dic_mot[2])):
            if ef_mot not in ef_dict_com_par:
                ef_dict_com_par.append(ef_mot)
        # Entités du dictionnaire qui englobent des entités de Spotlight
        if ((ef_mot[1] == dic_mot[1]) and (ef_mot[2] < dic_mot[2])) or ((ef_mot[1] > dic_mot[1]) and (ef_mot[2] == dic_mot[2])) or ((ef_mot[1] > dic_mot[1]) and (ef_mot[2] < dic_mot[2])):
            if  ef_mot not in ef_dict_com_par:
                ef_dict_com_par.append(ef_mot)

In [40]:
len(ef_dict_com_par)

1765

In [39]:
ef_dict_com_test = []
for ef_mot in ef_offset:
    for dic_mot in dict_offsets:
        # Entités du dictionnaire qui représentent juste une partie des entités de Spotlight
        if ((ef_mot[1] == dic_mot[1]) and (ef_mot[2] > dic_mot[2])) or ((ef_mot[1] < dic_mot[1]) and (ef_mot[2] == dic_mot[2])) or ((ef_mot[1] < dic_mot[1]) and (ef_mot[2] > dic_mot[2])):
            if  ef_mot[0] not in [ent[0][0] for ent in ef_dict_com_test]:
                ef_dict_com_test.append((ef_mot, dic_mot))

In [40]:
import pandas as pd
offEF = [f"{off[0][1]} - {off[0][2]}" for off in ef_dict_com_test]
entEF = [ent[0][0] for ent in ef_dict_com_test]
offTm = [f"{off[1][1]} - {off[1][2]}" for off in ef_dict_com_test]
entTm = [ent[1][0] for ent in ef_dict_com_test]

dc = {'Offsets Entity Fishing':offEF, 'Entités Entity Fishing':entEF, 'Offsets Terminologie':offTm, 'Entités Terminologie':entTm}
df = pd.DataFrame(dc)
df.to_excel('EntityLinking_Terminology.xlsx')

In [35]:
len(ef_dict_com)

3164

In [36]:
len(ef_offset)

5415

### 3. Calcul de statistique de recouvrement 

In [37]:
ef_recouvrement_extact = (len(ef_dict_com) * 100) /len(ef_offset)
ef_recouvrement_extact

58.43028624192059

In [41]:
ef_recouvrement_extact = (len(ef_dict_com_par) * 100) /len(ef_offset)
ef_recouvrement_extact

32.59464450600185

## 3) Mots communs entre les entités de Spotlight et le dictionnaire

### 1. Extraction des entités de Spotlight avec les offsets correspondants

In [22]:
from nltk.tokenize.punkt import PunktSentenceTokenizer

sen_offsets = []
for start, end in PunktSentenceTokenizer().span_tokenize(text):
    sen_offsets.append((start, end))

In [24]:
import requests
results = []

for offset in sen_offsets:
    results.append((requests.post("http://localhost:2222/rest/annotate", 
                                  data=("text=" + text[offset[0]:offset[1]]).encode('UTF-8'), 
                                  headers={'Accept': 'application/json'}),
                    offset[0]))
    

In [11]:
results[0]

(<Response [200]>, 0)

In [12]:
# Il y a des resultats vides
results[28]

(<Response [200]>, 9444)

In [25]:
import json
dataSL = [(json.loads(result[0].text), result[1]) for result in results if result[0].text != ""]
len(dataSL)

2689

In [16]:
dataSL[0]

({'@text': ' \nFICHE MÉMO \nPrise en charge du patient présentant \nune lombalgie commune \n \n \nRAPPORT D’ÉLABORATION \nMars 2019  \nCe document n’est pas le texte de recommandations \n \n \n \n \n \n  Prise en charge du patient présentant une lombalgie commune \nLa  méthode  d’élaboration  des  fiches  mémo  est  une  méthode  pour  produire  des \nrecommandations ou messages-clés dans un temps court (6 mois environ) et dans un format \ncourt (recto-verso).',
  '@confidence': '0.5',
  '@support': '0',
  '@types': '',
  '@sparql': '',
  '@policy': 'whitelist',
  'Resources': [{'@URI': 'http://fr.dbpedia.org/resource/Lombalgie',
    '@support': '117',
    '@types': '',
    '@surfaceForm': 'lombalgie',
    '@offset': '57',
    '@similarityScore': '0.9999999999597549',
    '@percentageOfSecondRank': '0.0'},
   {'@URI': 'http://fr.dbpedia.org/resource/Lombalgie',
    '@support': '117',
    '@types': '',
    '@surfaceForm': 'lombalgie',
    '@offset': '220',
    '@similarityScore': '0.999

Et il y a aussi des résultas qui ne sont pas vides mais qui ne contiennent aucune ressource donc il faut faire un test pour s'assurer que **'Resources'** appartient à la liste des clés du dictionnaire.

In [17]:
dataSL[1]

({'@text': 'Les fiches mémo s’inscrivent dans un objectif d’amélioration de la qualité et de la sécurité des \nsoins.',
  '@confidence': '0.5',
  '@support': '0',
  '@types': '',
  '@sparql': '',
  '@policy': 'whitelist'},
 440)

In [26]:
SpLi_offsets = []
for dic in dataSL:
    if 'Resources' in dic[0].keys():
        for res in dic[0]['Resources']:
            SpLi_offsets.append((res['@surfaceForm'], int(res['@offset']) + dic[1], int(res['@offset']) + int(len(res['@surfaceForm']) + dic[1]), res['@URI']))

In [27]:
SpLi_offsets

[('lombalgie', 57, 66, 'http://fr.dbpedia.org/resource/Lombalgie'),
 ('lombalgie', 220, 229, 'http://fr.dbpedia.org/resource/Lombalgie'),
 ('santé', 617, 622, 'http://fr.dbpedia.org/resource/Santé'),
 ('annexe 1',
  1069,
  1077,
  'http://fr.dbpedia.org/resource/Annexe_I_de_la_CITES'),
 ('Preuve scientifique',
  1124,
  1143,
  'http://fr.dbpedia.org/resource/Preuve_scientifique'),
 ('méta-analyse', 1313, 1325, 'http://fr.dbpedia.org/resource/Méta-analyse'),
 ("l'absence",
  1999,
  2008,
  "http://fr.dbpedia.org/resource/L'Absence_(film,_2010)"),
 ("L'absence",
  2150,
  2159,
  "http://fr.dbpedia.org/resource/L'Absence_(film,_2010)"),
 ('Haute Autorité de Santé',
  2383,
  2406,
  'http://fr.dbpedia.org/resource/Haute_Autorité_de_santé'),
 ('Saint-Denis La Plaine',
  2485,
  2506,
  'http://fr.dbpedia.org/resource/Saint-Denis_(Seine-Saint-Denis)'),
 ('Haute Autorité de Santé',
  2618,
  2641,
  'http://fr.dbpedia.org/resource/Haute_Autorité_de_santé'),
 ('Haute Autorité de Santé',
 

In [26]:
len(SpLi_offsets)

4382

In [15]:
# Comme vous remarqué bien les offsets sont corrects
text[617:622]

'santé'

In [28]:
with open('SpLi_offsets.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(SpLi_offsets)))

In [21]:
import json

with open('SpLi_offsets.json', 'r', encoding='utf-8') as f:
    SpLi_offsets = json.loads(f.read())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### 2. Recherche des entités communs avec les mêmes offsets, des entités du dictionnaire qui représentent juste une partie des entités de Spotlight et des entités du dictionnaire qui englobent des entités de Spotlight :

In [29]:
spli_dict_com = []
for sp_mot in SpLi_offsets:
    for dic_mot in dict_offsets:
        # Entités communs avec les mêmes offsets
        if sp_mot[1] == dic_mot[1] and sp_mot[2] == dic_mot[2]:
            if sp_mot not in spli_dict_com:
                spli_dict_com.append(sp_mot[0])
                print(f"Spotlight: {sp_mot} ---- Terminologie: {dic_mot}")

Spotlight: ('Preuve scientifique', 1124, 1143, 'http://fr.dbpedia.org/resource/Preuve_scientifique') ---- Terminologie: ('Preuve scientifique', 1124, 1143)
Spotlight: ('lombalgie', 5208, 5217, 'http://fr.dbpedia.org/resource/Lombalgie') ---- Terminologie: ('lombalgie', 5208, 5217)
Spotlight: ('lombalgie', 5208, 5217, 'http://fr.dbpedia.org/resource/Lombalgie') ---- Terminologie: ('lombalgie', 5208, 5217)
Spotlight: ('AINS', 6675, 6679, 'http://fr.dbpedia.org/resource/Anti-inflammatoire_non_stéroïdien') ---- Terminologie: ('AINS', 6675, 6679)
Spotlight: ('Intervalle de confiance', 6982, 7005, 'http://fr.dbpedia.org/resource/Intervalle_de_confiance') ---- Terminologie: ('Intervalle de confiance', 6982, 7005)
Spotlight: ('IRM', 7007, 7010, 'http://fr.dbpedia.org/resource/IRM_cardiaque') ---- Terminologie: ('IRM', 7007, 7010)
Spotlight: ('IRM', 7007, 7010, 'http://fr.dbpedia.org/resource/IRM_cardiaque') ---- Terminologie: ('IRM', 7007, 7010)
Spotlight: ('National Institute for Health and C

In [30]:
len(spli_dict_com)

2403

In [17]:
spli_dict_com

[['santé', 617, 622],
 ['Preuve scientifique', 1124, 1143],
 ['méta-analyse', 1313, 1325],
 ['Haute Autorité de Santé', 2383, 2406],
 ['Haute Autorité de Santé', 2618, 2641],
 ['Haute Autorité de Santé', 2659, 2682],
 ['lombalgie', 5208, 5217],
 ['AINS', 6675, 6679],
 ['CNAM', 6724, 6728],
 ['DGOS', 6775, 6779],
 ['EVA', 6825, 6828],
 ['Intervalle de confiance', 6982, 7005],
 ['IRM', 7007, 7010],
 ['santé', 7145, 7150],
 ['TENS', 7217, 7221],
 ['CNAM', 7513, 7517],
 ['DGOS', 7567, 7571],
 ['santé', 7798, 7803],
 ['français', 8317, 8325],
 ['santé publique', 8613, 8627],
 ['santé', 8634, 8639],
 ['médecin', 9478, 9485],
 ['français', 9499, 9507],
 ['lombalgie', 9534, 9543],
 ['santé', 9610, 9615],
 ['lombalgie', 9768, 9777],
 ['France', 10173, 10179],
 ['santé', 10213, 10218],
 ['santé', 10473, 10478],
 ['France', 11346, 11352],
 ['nuelles', 12923, 12930],
 ['neurostimulation', 13015, 13031],
 ['TENS', 13108, 13112],
 ['acupuncture', 13115, 13126],
 ['lombalgie', 13738, 13747],
 ['lomba

In [9]:
relatedTerms = set(ef_dict_com + spli_dict_com)
with open('relatedTerms.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(relatedTerms)))

In [42]:
spli_dict_com_par = []
for sp_mot in SpLi_offsets:
    for dic_mot in dict_offsets:
        # Entités du dictionnaire qui représentent juste une partie des entités de Spotlight
        if sp_mot[1] == dic_mot[1] and sp_mot[2] > dic_mot[2] or sp_mot[1] < dic_mot[1] and sp_mot[2] == dic_mot[2] or sp_mot[1] < dic_mot[1] and sp_mot[2] > dic_mot[2]:
            if sp_mot not in spli_dict_com_par:
                spli_dict_com_par.append(sp_mot)
        # Entités du dictionnaire qui englobent des entités de Spotlight
        if sp_mot[1] == dic_mot[1] and sp_mot[2] < dic_mot[2] or sp_mot[1] > dic_mot[1] and sp_mot[2] == dic_mot[2] or sp_mot[1] > dic_mot[1] and sp_mot[2] < dic_mot[2]:
            if  sp_mot not in spli_dict_com_par:
                spli_dict_com_par.append(sp_mot)

In [43]:
len(spli_dict_com_par)

685

#### 3. Calcul de statistique de recouvrement

In [47]:
Sl_recouvrement_exact = (len(spli_dict_com) * 100) /len(SpLi_offsets)
Sl_recouvrement_exact

54.83797352806938

In [46]:
Sl_recouvrement_partiel = (len(spli_dict_com_par) * 100) /len(SpLi_offsets)
Sl_recouvrement_partiel

15.632131446827932

In [14]:
sp_dict_com_test = []
for sp_mot in SpLi_offsets:
    for dic_mot in dict_offsets:
        # Entités communs avec les mêmes offsets
        if sp_mot[1] == dic_mot[1] and sp_mot[2] == dic_mot[2]:            
            sp_dict_com_test.append((sp_mot, dic_mot))

In [15]:
import pandas as pd
offSl = [f"{off[0][1]} - {off[0][2]}" for off in sp_dict_com_test]
entSl = [ent[0][0] for ent in sp_dict_com_test]
offTm = [f"{off[1][1]} - {off[1][2]}" for off in sp_dict_com_test]
entTm = [ent[1][0] for ent in sp_dict_com_test]

dc = {'Offsets Spotlight':offSl, 'Entités Spotlight':entSl, 'Offsets Terminologie':offTm, 'Entités Terminologie':entTm}
df = pd.DataFrame(dc)
df.to_excel('EntityLinking_exact.xlsx')